# Notebook for the creation of the disk lsh hashes for both datasets

Sheet that converts the extracted data from the data/chosen_data folder to hashes that will be stored in data/hashed_data/disk

In [22]:
# Importing nescessary modules
import os
import shutil

from schemes.disk_lsh import DiskLSH
from schemes.experiments import hashing
from utils import metafile_handler as mfh

import timeit as ti
from tqdm import tqdm 

from multiprocessing import Pool

In [23]:
# Declaring global variables:

SHOULD_DELETE_OLD_FILES= False

OUTPUT_FOLDER_PORTO = "../data/hashed_data/disk/porto/"

PORTO_DATA = "../data/chosen_data/porto/"

P_MAX_LON = -8.57
P_MIN_LON = -8.66
P_MAX_LAT = 41.19
P_MIN_LAT = 41.14

## Porto LSH

Beginning with the porto set

In [24]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    for filename in os.listdir(OUTPUT_FOLDER_PORTO):
        file_path = os.path.join(OUTPUT_FOLDER_PORTO, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Failed to remove %s. Reason: %s" % (file_path, e))

In [25]:
# Create a disk-based LSH object over Porto

layers = 4
diameter = 1.5
num_disks = 50
meta_file = "../data/chosen_data/porto/META-1000.txt"

DiskPorto = DiskLSH("Porto D1", P_MIN_LAT, P_MAX_LAT, P_MIN_LON, P_MAX_LON, num_disks, layers, diameter, meta_file, PORTO_DATA)

In [26]:
# Generating the disk-based LSH objects hashes and saving them to file

hashes = DiskPorto.compute_dataset_hashes_with_KD_tree()

for key in hashes:
    with open(f'{OUTPUT_FOLDER_PORTO}{key}.txt', 'w') as file:
        for hash in hashes[key]:
            file.write("%s\n" % hash)
        file.close

# Copying meta_files as well
meta_files = mfh.get_meta_files(PORTO_DATA)

for filename in meta_files:
    shutil.copy(PORTO_DATA + filename, OUTPUT_FOLDER_PORTO)

In [27]:
# Cell for methdos use to measure the hashing performance efficiency, utilises parallell programming for optimal measurements
layers = 4
diameter = 1.5
num_disks = 50
meta_file = "../data/chosen_data/porto/META-200.txt"

def fun_wrapper_naive(x):
    disk = DiskLSH("Porto D1", P_MIN_LAT, P_MAX_LAT, P_MIN_LON, P_MAX_LON, num_disks, layers, diameter, meta_file, PORTO_DATA)
    return disk.measure_hash_computation(1,1)[0]

def fun_wrapper_quadrants(x):
    disk = DiskLSH("Porto D1", P_MIN_LAT, P_MAX_LAT, P_MIN_LON, P_MAX_LON, num_disks, layers, diameter, meta_file, PORTO_DATA)
    return disk.measure_hash_computation_with_quad_tree(1,1)[0]

def fun_wrapper_KD_tree(x):
    disk = DiskLSH("Porto D1", P_MIN_LAT, P_MAX_LAT, P_MIN_LON, P_MAX_LON, num_disks, layers, diameter, meta_file, PORTO_DATA)
    return disk.compute_dataset_hashes_with_KD_tree(1,1)[0]

#runtimes, number = DiskPorto.measure_hash_computation_with_KD_tree(number=1, repeat=10)
#print(runtimes)
#print(f"Ran the hash computation {len(runtimes)} times. Fastest runtime: {min(runtimes)}s for {number} hashes")

# Measuring run-times of hash generation
The cells below are created to measure the time-efficiency of the hash computation


In [28]:
#Cell for measrung DiskLSH hash generation times for all methods in both datasets - using coordiante hashes
import pandas as pd
from itertools import chain

output_folder = "../code/schemes/experiments/runtimes/"
file_name = "hashing_runtimes_disk_lsh.csv"

hashing_map = {
    "porto_naive" : hashing.fun_wrapper_p_naive,
    "porto_quadrants" : hashing.fun_wrapper_p_quadrants,
    "porto_kd_tree" : hashing.fun_wrapper_p_KD_tree,
}

config = {
    "porto" : [1000, 60, 4, 2.2]
}

runs = 10

df = pd.DataFrame(columns=[f"Run_{run+1}" for run in range(runs)])

for key in hashing_map.keys():
    with Pool() as pool:
        result = pool.map(hashing_map[key], [config[key.split("_")[0]] for _ in range(runs)])
        df.loc[key] = list(chain.from_iterable(result))

df.to_csv(os.path.join(output_folder, file_name))